In [1]:
import os, json
import numpy as np
# from argparse import ArgumentParser
from tqdm import tqdm
from collections import defaultdict
from pprint import pprint
# import stanza
# import ipdb

In [2]:
def read_docs(path):
    all_objs = []
    
    for file in os.listdir(path):
        ## 检查文件是不是json格式
        if not file.endswith("json"):
            continue
        
        try:
            with open(os.path.join(path, file)) as fp:
                obj = json.load(fp)
        except:
            continue
        all_objs.append(obj)
    
    return all_objs

In [3]:
in_folder = './annotation/'
objs = read_docs(in_folder)

In [8]:
def get_split(objs, split):
    
    with open(os.path.join(split, "train.txt")) as fp:
        lines = fp.readlines()
        train_doc_ids = set([l.strip() for l in lines])
        
    with open(os.path.join(split, "dev.txt")) as fp:
        lines = fp.readlines()
        dev_doc_ids = set([l.strip() for l in lines])
        
    with open(os.path.join(split, "test.txt")) as fp:
        lines = fp.readlines()
        test_doc_ids = set([l.strip() for l in lines])

    train_objs = []
    dev_objs = []
    test_objs = []
    
    for obj in objs:
        if obj["sourcefile"] in train_doc_ids:
            train_objs.append(obj)
        if obj["sourcefile"] in dev_doc_ids:
            dev_objs.append(obj)
        if obj["sourcefile"] in test_doc_ids:
            test_objs.append(obj)
    
    train_objs.sort(key=lambda x: x["sourcefile"])
    dev_objs.sort(key=lambda x: x["sourcefile"])
    test_objs.sort(key=lambda x: x["sourcefile"])
    
    return train_objs, dev_objs, test_objs

In [9]:
split = './split1/'
train_objs, dev_objs, test_objs = get_split(objs, split)

In [10]:
print(len(train_objs),len(dev_objs),len(test_objs))

701 149 149


In [14]:
data = []
n_drop_event = 0
n_drop_arg = 0
with open("token_map.json") as fp:
    token_map = json.load(fp)

with open("seg_map.json") as fp:
    seg_map = json.load(fp)

In [12]:
obj = train_objs[0]
text_tokens = [[obj["content"][si:ei] for si, ei in sent] for sent in token_map[obj["sourcefile"]]]
'''
token_map:列表形式，第一层为每个句子，第二层为每个句子内部分词
text_tokens:列表形式，第一层代表句子，第二层为句子分词
'''

In [48]:
n_sent = len(text_tokens)
print(n_sent)
sent_lens = [len(s) for s in text_tokens]
print(sent_lens)

12
[25, 44, 25, 25, 33, 9, 19, 10, 15, 15, 19, 3]


In [49]:
sents = seg_map[obj["sourcefile"]]
n_seg = len(sents)
print(sents,n_seg)

[[0, 12]] 1


In [50]:
entity_mentions = defaultdict(list)
event_mentions = defaultdict(list)

In [51]:
sent_offsets = token_map[obj["sourcefile"]]

In [52]:
seg_offsets = []
'''
将所有句子以及分词整合到一个列表中
'''
for si, ei in sents:
            segs = []
            for i in range(si, ei):
                segs.extend(sent_offsets[i])
            seg_offsets.append(segs)

In [62]:
obj['cyberevent']

{'hopper': [{'relation': 'Same',
   'index': 0,
   'events': [{'nugget': {'startOffset': 273,
      'index': 'T8',
      'endOffset': 290,
      'text': 'had been accessed'},
     'subtype': 'Databreach',
     'argument': [{'startOffset': 262,
       'index': 'T10',
       'endOffset': 271,
       'role': {'type': 'Compromised-Data'},
       'type': 'PII',
       'text': 'addresses'},
      {'startOffset': 238,
       'index': 'T9',
       'endOffset': 257,
       'role': {'type': 'Compromised-Data'},
       'type': 'PII',
       'text': 'their phone numbers'},
      {'startOffset': 215,
       'index': 'T11',
       'endOffset': 226,
       'role': {'type': 'Compromised-Data'},
       'type': 'Data',
       'text': 'information'},
      {'startOffset': 189,
       'index': 'T12',
       'endOffset': 194,
       'role': {'type': 'Victim'},
       'external_reference': {'wikidataid': 'Q467'},
       'type': 'Person',
       'text': 'women'}],
     'index': 'E3',
     'realis': 'Actual',

In [61]:
hopper = obj['cyberevent']['hopper'][0]
hopper

{'relation': 'Same',
 'index': 0,
 'events': [{'nugget': {'startOffset': 273,
    'index': 'T8',
    'endOffset': 290,
    'text': 'had been accessed'},
   'subtype': 'Databreach',
   'argument': [{'startOffset': 262,
     'index': 'T10',
     'endOffset': 271,
     'role': {'type': 'Compromised-Data'},
     'type': 'PII',
     'text': 'addresses'},
    {'startOffset': 238,
     'index': 'T9',
     'endOffset': 257,
     'role': {'type': 'Compromised-Data'},
     'type': 'PII',
     'text': 'their phone numbers'},
    {'startOffset': 215,
     'index': 'T11',
     'endOffset': 226,
     'role': {'type': 'Compromised-Data'},
     'type': 'Data',
     'text': 'information'},
    {'startOffset': 189,
     'index': 'T12',
     'endOffset': 194,
     'role': {'type': 'Victim'},
     'external_reference': {'wikidataid': 'Q467'},
     'type': 'Person',
     'text': 'women'}],
   'index': 'E3',
   'realis': 'Actual',
   'type': 'Attack'},
  {'nugget': {'startOffset': 324,
    'index': 'T6',
  

In [75]:
for event in hopper["events"]:
    trigger = event['nugget']
    print(event)

{'nugget': {'startOffset': 273, 'index': 'T8', 'endOffset': 290, 'text': 'had been accessed'}, 'subtype': 'Databreach', 'argument': [{'startOffset': 262, 'index': 'T10', 'endOffset': 271, 'role': {'type': 'Compromised-Data'}, 'type': 'PII', 'text': 'addresses'}, {'startOffset': 238, 'index': 'T9', 'endOffset': 257, 'role': {'type': 'Compromised-Data'}, 'type': 'PII', 'text': 'their phone numbers'}, {'startOffset': 215, 'index': 'T11', 'endOffset': 226, 'role': {'type': 'Compromised-Data'}, 'type': 'Data', 'text': 'information'}, {'startOffset': 189, 'index': 'T12', 'endOffset': 194, 'role': {'type': 'Victim'}, 'external_reference': {'wikidataid': 'Q467'}, 'type': 'Person', 'text': 'women'}], 'index': 'E3', 'realis': 'Actual', 'type': 'Attack'}
{'nugget': {'startOffset': 324, 'index': 'T6', 'endOffset': 332, 'text': 'breached'}, 'subtype': 'Databreach', 'argument': [{'startOffset': 300, 'index': 'T7', 'endOffset': 319, 'role': {'type': 'Attacker'}, 'type': 'Person', 'text': 'the member 

In [77]:
tri_seg_id = -1
for i, sent_offsets in enumerate(seg_offsets):
    
    if sent_offsets[0][0] <= trigger['startOffset'] and trigger['endOffset'] <= sent_offsets[-1][1]:
        tri_seg_id = i
        break

In [97]:
tri_start_id = -1
tri_end_id = -1
for i, token_offsets in enumerate(seg_offsets[tri_seg_id]):
    '''
    i表示第i个词在列表中的位置

    '''
    if token_offsets[0] == trigger["startOffset"]:
        tri_start_id = i
        # print(tri_start_id)
        ## 从第i个元素开始到最后，寻找end
        for j in range(i, len(seg_offsets[tri_seg_id])):
            if seg_offsets[tri_seg_id][j][1] == trigger["endOffset"]:
                tri_end_id = j + 1
        break

In [105]:
arguments = []
if 'argument' in event:
    for arg in event['argument']:
        print(arg)
        arg_seg_id = -1
        for i, sent_offsets in enumerate(seg_offsets):
            if sent_offsets[0][0] <= arg['startOffset'] and arg['endOffset'] <= sent_offsets[-1][1]:
                arg_seg_id = i
                break
        if arg_seg_id == -1 or tri_seg_id != arg_seg_id:
                            n_drop_arg += 1
                            continue

{'startOffset': 801, 'index': 'T20', 'endOffset': 818, 'role': {'type': 'Attacker'}, 'type': 'Person', 'text': 'a member of staff'}
{'startOffset': 845, 'index': 'T22', 'endOffset': 860, 'role': {'type': 'Compromised-Data'}, 'type': 'PII', 'text': 'patient records'}


In [108]:
f"{obj['sourcefile'][:-4]}_Seg{tri_seg_id}_Ent{len(entity_mentions[tri_seg_id])}"

'10001_Seg0_Ent0'

In [109]:
def convert_format(objs, token_map, seg_map):
    '''
    objs:存储json文档的list
    token_map:分句子
    '''
    data = []
    n_drop_event = 0
    n_drop_arg = 0
    for obj in tqdm(objs, ncols=100):
        
        text_tokens = [[obj["content"][si:ei] for si, ei in sent] for sent in token_map[obj["sourcefile"]]]
        n_sent = len(text_tokens) # 多少句
        sent_lens = [len(s) for s in text_tokens] # 每句长度
        
        sents = seg_map[obj["sourcefile"]]
        n_seg = len(sents)
        
        entity_mentions = defaultdict(list)
        event_mentions = defaultdict(list)
        sent_offsets = token_map[obj["sourcefile"]]
        seg_offsets = []
        for si, ei in sents:
            segs = []
            for i in range(si, ei):
                segs.extend(sent_offsets[i])
            seg_offsets.append(segs)
                
        for hopper in obj['cyberevent']['hopper']:## [0]?
            for event in hopper["events"]:
                trigger = event['nugget']
                tri_seg_id = -1
                for i, sent_offsets in enumerate(seg_offsets):
                    if sent_offsets[0][0] <= trigger['startOffset'] and trigger['endOffset'] <= sent_offsets[-1][1]:
                        tri_seg_id = i
                        break
                if tri_seg_id == -1:
                    n_drop_event += 1
                    continue
                    
                tri_start_id = -1
                tri_end_id = -1
                for i, token_offsets in enumerate(seg_offsets[tri_seg_id]):
                    if token_offsets[0] == trigger["startOffset"]:
                        tri_start_id = i
                        for j in range(i, len(seg_offsets[tri_seg_id])):
                            if seg_offsets[tri_seg_id][j][1] == trigger["endOffset"]:
                                tri_end_id = j + 1
                        break
                
                if tri_start_id == -1 or tri_end_id == -1:
                    if obj['content'][trigger["startOffset"]:trigger["endOffset"]] != trigger["text"] and obj['content'][trigger["startOffset"]-1:trigger["endOffset"]-1] == trigger["text"]:
                        tri_start_id = -1
                        tri_end_id = -1
                        for i, token_offsets in enumerate(seg_offsets[tri_seg_id]):
                            if token_offsets[0] == trigger["startOffset"]-1:
                                tri_start_id = i
                                for j in range(i, len(seg_offsets[tri_seg_id])):
                                    if seg_offsets[tri_seg_id][j][1] == trigger["endOffset"]-1:
                                        tri_end_id = j + 1
                                break
                        
                        if tri_start_id == -1 or tri_end_id == -1:
                            n_drop_event += 1
                            continue
                    else:
                        n_drop_event += 1
                        continue
                    
                arguments = []
                if 'argument' in event:
                    for arg in event['argument']:
                        arg_seg_id = -1
                        for i, sent_offsets in enumerate(seg_offsets):
                            if sent_offsets[0][0] <= arg['startOffset'] and arg['endOffset'] <= sent_offsets[-1][1]:
                                arg_seg_id = i
                                break
                        if arg_seg_id == -1 or tri_seg_id != arg_seg_id:
                            n_drop_arg += 1
                            continue
                            
                        start_id = -1
                        end_id = -1
                        for i, token_offsets in enumerate(seg_offsets[arg_seg_id]):
                            if token_offsets[0] == arg["startOffset"]:
                                start_id = i
                                for j in range(i, len(seg_offsets[arg_seg_id])):
                                    if seg_offsets[arg_seg_id][j][1] == arg["endOffset"]:
                                        end_id = j + 1
                                break

                        if start_id == -1 or end_id == -1:
                            if obj['content'][arg["startOffset"]:arg["endOffset"]] != arg["text"] and obj['content'][arg["startOffset"]-1:arg["endOffset"]-1] == arg["text"]:
                                start_id = -1
                                end_id = -1
                                
                                for i, token_offsets in enumerate(seg_offsets[arg_seg_id]):
                                    if token_offsets[0] == arg["startOffset"]-1:
                                        start_id = i
                                        for j in range(i, len(seg_offsets[arg_seg_id])):
                                            if seg_offsets[arg_seg_id][j][1] == arg["endOffset"]-1:
                                                end_id = j + 1
                                        break
                                
                                if start_id == -1 or end_id == -1:
                                    n_drop_arg += 1
                                    continue
                            else:
                                n_drop_arg += 1
                                continue
                        
                        entity_id = f"{obj['sourcefile'][:-4]}_Seg{tri_seg_id}_Ent{len(entity_mentions[tri_seg_id])}"
                        entity = {
                            "id": entity_id, 
                            "text": arg["text"], 
                            "entity_type": "Entity", 
                            "start": start_id, 
                            "end": end_id, 
                        }
                        entity_mentions[tri_seg_id].append(entity)
                        
                        argument = {
                            "entity_id": entity_id, 
                            "role": arg["role"]["type"], 
                            "text": arg["text"], 
                            "start": start_id, 
                            "end": end_id, 
                        }
                        arguments.append(argument)
                    arguments.sort(key=lambda x: (x["start"], x["end"]))
                
                event_ = {
                    "id": f"{obj['sourcefile'][:-4]}_Seg{tri_seg_id}_Evt{len(event_mentions[tri_seg_id])}",
                    "event_type": f"{event['type']}:{event['subtype']}",
                    "trigger": {
                        "text": trigger["text"], 
                        "start": tri_start_id, 
                        "end": tri_end_id, 
                    },
                    "arguments": arguments, 
                }
                event_mentions[tri_seg_id].append(event_)
                
        for i in range(n_seg):
            entity_mentions_ = entity_mentions[i]
            event_mentions_ = event_mentions[i]
            
            entity_mentions_ = sorted(entity_mentions_, key=lambda x: (x["start"], x["end"]))
            event_mentions_ = sorted(event_mentions_, key=lambda x: (x["trigger"]["start"], x["trigger"]["end"]))
            tokens = [t for j in range(sents[i][0], sents[i][1]) for t in text_tokens[j]]
            
            dt = {
                "doc_id": obj['sourcefile'][:-4], 
                "wnd_id": f"{obj['sourcefile'][:-4]}_{i}", 
                "text": obj["content"][seg_offsets[i][0][0]:seg_offsets[i][-1][1]], 
                "tokens": tokens, 
                "event_mentions": event_mentions_, 
                "entity_mentions": entity_mentions_, 
                "lang": "en", 
            }
            data.append(dt)
        
    print(f"Number of dropped events: {n_drop_event}")
    print(f"Number of dropped arguments: {n_drop_arg}")
            
    return data

In [110]:
train_data = convert_format(train_objs, token_map, seg_map)

100%|████████████████████████████████████████████████████████████| 701/701 [00:00<00:00, 890.50it/s]

Number of dropped events: 4
Number of dropped arguments: 41


In [113]:
train_data[0]

{'doc_id': '10001',
 'wnd_id': '10001_0',
 'text': 'POLICE have launched an investigation after a number of patients have had their personal details accessed by a member of staff at Crosshouse Hospital. Those affected, believed to be mainly women, were notified that information, including their phone numbers and addresses, had been accessed and that the member of staff who breached their privacy had ‘no legitimate clinical or administrative requirement’. The breach came from within information held in the Radiology Information System and patients affected had all recently been referred for an X-ray.   A member of staff has been \'excluded from work\' following the breach which took place between April and September of this year.  Dr Alison Graham, Medical Director for the NHS Ayrshire & Arran said: "NHS Ayrshire & Arran has been made aware of a member of staff inappropriately accessing patient records. This individual is currently excluded from work." We are currently investigating and

In [112]:
def get_statistics(data):
    event_type_count = defaultdict(int)
    role_type_count = defaultdict(int)
    doc_ids = set()
    max_len = 0
    for dt in data:
        max_len = max(max_len, len(dt["tokens"]))
        doc_ids.add(dt["doc_id"])
        for event in dt["event_mentions"]:
            event_type_count[event["event_type"]] += 1
            for argument in event["arguments"]:
                role_type_count[argument["role"]] += 1
    
    print(f"# of Instances: {len(data)}")
    print(f"# of Docs: {len(doc_ids)}")
    print(f"Max Length: {max_len}")
    print(f"# of Event Types: {len(event_type_count)}")
    print(f"# of Events: {sum(event_type_count.values())}")
    print(f"# of Role Types: {len(role_type_count)}")
    print(f"# of Arguments: {sum(role_type_count.values())}")
    # pprint(event_type_count)
    print()
print("Train")
get_statistics(train_data)

Train
# of Instances: 1047
# of Docs: 701
Max Length: 463
# of Event Types: 5
# of Events: 5980
# of Role Types: 26
# of Arguments: 15869

